In [2]:
from utils.ML2.utils import *
import warnings
warnings.filterwarnings('ignore')

# 7. Application

## 7.1 Payload/Tool parameter identification

In robotics, the identification of mass, is crucial for ensuring accurate and safe operation,
particularly in tasks that involve dynamic interactions with the environment like pick
and place. Traditional methods of measuring the weight of the load often require direct
physical measurements, which can be both time-consuming and impractical in automated
systems where multiple types of payloads are involved. This application of a the devel-
oped model offers a significant advantage by enabling the indirect estimation of these
parameters through the analysis of predicted forces. This increases the adaptability of
the robot to various tools and payloads without the need for frequent recalibration or
manual measurements or a costly FTS.

Another particularly valuable use case for this technology is in detecting situations where a
payload, after being picked up, might slip down. Such a slip can be quickly detected by the
virtual sensor. Without this detection capability, the robot could experience unexpected
torques and forces, leading to a sudden jerk, which could compromise the precision of
the task or trigger false collision, resulting in lost time and money. By identifying these
changes in real-time, the system can adjust its operations to compensate, ensuring smooth
and safe handling of the payload.

### 7.1.1 Implementation

The estimation of mass using a virtual joint torque sensor is done using the estimate of forces acting on the TCP. The forces estimated by the virtual sensor, in the $X$, $Y$, and $Z$ directions, can be used to compute the overall magnitude of the force vector, which, when divided by the gravitational constant, provides an estimate of the tool's mass.

In a static or quasi-static condition, the forces measured by the virtual joint torque sensor are primarily due to the weight of the tool and any attached payload. The relationship between the force components and the mass of the tool can be expressed as:

\begin{equation}
|\vec{F}| = \sqrt{F_x^2 + F_y^2 + F_z^2}\tag{7.1}
\end{equation}


where:

- $F_x$, $F_y$, and $F_z$ are the force components in the $X$, $Y$, and $Z$ directions, respectively.
- $|\vec{F}|$ is the magnitude of the resultant force vector.

Given that the force due to gravity is $F = m \cdot g$, where $g$ is the acceleration due to gravity (approximately $9.81 \, \text{m/s}^2$), the mass $m$ can be estimated as:

\begin{equation}m = \frac{|\vec{F}|}{g}\tag{7.2}\end{equation}

To ensure accuracy, measurements are made over a period of 200 ms resulting in 100 data
points, and the average of the calculated mass values is taken to provide a more reliable
estimate. This approach helps to reduce the impact of sensor noise, and compensate for
any variability in the conditions during the measurements, leading to a more accurate
estimation of the tool’s mass. The above equation assumes that the forces measured are
primarily due to gravity, and thus the resultant force vector provides a direct indication
of the tool’s mass.

To validate the performance of the model in this application, an eccentric mass as shown
in <a href="#eccentric_tool">Figure 7.1</a> was attached to the robot and the robot was moved along a generated
trajectory as described in [Section 6.1](./6.0_Machine_Learning_based_approach_for_entire_robot.ipynb#6.1.-Data-Collection). A velocity scaling of 5% was applied so as to maintain a
quasi-static condition.

<figure style="text-align: center;">
    <img src="./utils/ML2/application/eccentrictool.jpeg" id="eccentric_tool" style="width:30%; margin: auto;">
    <figcaption align="center"> <b>Figure 7.1.: </b> Eccentric tool mounted on LARA8 </figcaption>
</figure>


In [3]:
y_max = torch.load('utils/ML2/models/y_max.pt',map_location=torch.device('cpu'))
y_min = torch.load('utils/ML2/models/y_min.pt',map_location=torch.device('cpu'))

dataset = CustomDatasetFromCSV(csv_path = 'utils/ML2/application/lara8_0.5.csv', mode="test")
test_dataloader = DataLoader(dataset, batch_size=100, shuffle=False)

model = torch.jit.load('utils/ML2/models/model.pt', map_location=torch.device('cpu'))
model.eval()

pred_list = []
with torch.no_grad():    
    for i, (X,y) in enumerate(tqdm(test_dataloader)):
        pred = model(X)
        #denormalize predictions
        pred = (pred+1)/2* (y_max - y_min) + y_min
        pred_list.append(pred)
# Denormalize the data
y_truth = ((test_dataloader.dataset.y + 1) / 2 * (y_max - y_min) + y_min).cpu().numpy()
pred_list = torch.cat(pred_list).cpu().numpy()

y_truth = y_truth[116:]

# Example usage of your specific mass calculation
Fx = pred_list[0:100, 0]
Fy = pred_list[0:100, 1]
Fz = pred_list[0:100, 2]

FTx = y_truth[0:100, 0]
FTy = y_truth[0:100, 1]
FTz = y_truth[0:100, 2]

mass_estimate = np.sqrt(Fx**2 + Fy**2 + Fz**2).mean() / 9.81
mass_actual = np.sqrt(FTx**2 + FTy**2 + FTz**2).mean() / 9.81
print("Mass estimated from model predictions: ", mass_estimate.item(),"kg")
print("Mass estimated from FTS measurements: ", mass_actual.item(),"kg")

  0%|          | 0/1 [00:00<?, ?it/s]

Mass estimated from model predictions:  2.7590842057440987 kg
Mass estimated from FTS measurements:  2.7074553307893927 kg


The model estimated the mass to be **2.71kg** whereas the mass estimated from FTS
measurements was **2.76kg**. <a href="#table7.1">Table 7.1</a> shows the model estimation on a number of different
payloads. The results demonstrates that the model can be used to estimate the mass of a
tool or object attached to the robot’s TCP reliably. This application can be extended to
calculate the payload’s center of gravity offsets using the model’s torque predictions.
<a id="table7.1"></a> 
<table>
    <caption style="caption-side:bottom; text-align:center;"><b>Table 7.1.:</b> Comparison of Estimated and Actual Masses with Absolute Errors</caption>
    <tr>
        <th>Payload</th>
        <th>Estimated Mass (kg)</th>
        <th>Actual Mass (kg)</th>
        <th>Absolute Error (kg)</th>
    </tr>
    <tr>
        <td>Payload 1</td>
        <td>2.71</td>
        <td>2.76</td>
        <td>0.04</td>
    </tr>
    <tr>
        <td>Payload 2</td>
        <td>1.5554</td>
        <td>1.5223</td>
        <td>0.0331</td>
    </tr>
    <tr>
        <td>Payload 3</td>
        <td>7.2118</td>
        <td>7.1823</td>
        <td>0.0295</td>
    </tr>
</table>

